Class 0
case 1 - Laz yes classifier:
- dsm with 0
- dsm without 0  
- compare 

case 2: Laz not classified 
- get the number of classes and unique values [x]
- dsm with 0 
- dsm without 0 
- compare 


no difference we decided to go with last return and max 

In [8]:
import os 
from glob import glob
import pdal 
import numpy as np 
import time 
from concurrent.futures import ProcessPoolExecutor
from uvars import eba_laz_dir, eba_tif_dir,eba_lazclf_dir,transect_names 

def pdal_quick_info(laz_path):
    reader = pdal.Reader(laz_path) # sensitive to resolution, can pass argument here 
    pipeline = reader.pipeline()
    qi = pipeline.quickinfo 
    return qi,pipeline


def pdal_pc_classes(pipeline):
    pipeline.execute()
    arrays = pipeline.arrays
    if len(arrays) == 0:
        print("No point data found in the file.")
    
    classification_values = arrays[0]['Classification']
    unique_classes = np.unique(classification_values)
    print(f"Unique classifications: {unique_classes}")
    return unique_classes

def clf_pointcloud(laz, lazo, method="smrf"):
    ti = time.perf_counter()
    print("""Classifies ground points in a point cloud and saves to LAZ.""")
    lazo = lazo.replace('.laz', f'_{method}.laz')

    if os.path.isfile(lazo):
        print(f"file aready created {lazo}")
        return lazo 
    
    pipeline = pdal.Reader(laz)
    pipeline |= pdal.Filter.expression(expression="Classification != 7")
    pipeline |= pdal.Filter.assign(assignment="Classification[:]=0")
    if method == 'smrf':
        pipeline |= pdal.Filter.smrf()
    elif method == 'csf':
        pipeline |= pdal.Filter.csf()
    
    pipeline |= pdal.Writer.las(filename=lazo, compression="laszip")
    #https://pdal.io/en/latest/stages/writers.las.html
    pipeline.execute()
    print(f"Reclassified point cloud saved to \n{lazo}")
    tf = time.perf_counter() - ti 
    print(f"run.time = {tf/60} min(s)")
    return lazo

compare mean, and max here too

In [9]:
import os
import time
import pdal

def generate_dsm_by_range(laz, tif, res=10, outfn="mean",lc=0,hc=4):
    ti = time.perf_counter()

    # Define the output file name
    tif = tif.replace('.tif', f"_rangeDSM{str(res)}_{outfn}_{lc}{hc}.tif")

    # Check if the file already exists
    if os.path.isfile(tif):
        print(f"File already exists...\n{tif}")
        return tif 

    print(f"Extracting points (classes {lc} to {hc}) and generating a DSM.")
    print(f"Generating DSM: {laz} \n-> {tif}")

    # Define the PDAL pipeline
    pipeline = pdal.Reader(laz)
    # Keep classifications 0 to 5 for DSM
    pipeline |= pdal.Filter.range(limits=f"Classification[{lc}:{hc}]")
    # Write to GeoTIFF with specified resolution and compression
    pipeline |= pdal.Writer.gdal(
        filename=tif,
        gdalopts="tiled=yes,compress=deflate",
        nodata=-9999,
        output_type=outfn,
        resolution=res
    )

    # Execute the pipeline
    pipeline.execute()
    tf = time.perf_counter() - ti 
    print(f"DSM saved to {tif}")
    print(f"Run time = {tf / 60:.2f} minute(s)")
    return tif

def generate_dsm_by_hag(lazo, dsm_tif, res, outfn):
    ti = time.perf_counter() 
    """Extracts highest points and generates a DSM."""
    # Define the output file name
    tif = dsm_tif = dsm_tif.replace('.tif', f"_hagDSM{str(res)}_{outfn}.tif")

    # Check if the file already exists
    if os.path.isfile(tif):
        print(f"File already exists...\n{tif}")
        return tif 

    pipeline = pdal.Reader(lazo)
    pipeline |= pdal.Filter.outlier(method="radius", radius=1.0, min_k=3)  # Remove high outliers
    #pipeline |= pdal.Filter.farthestneighbour()
    pipeline |= pdal.Filter.hag_delaunay()  # Use HAG Delaunay triangulation for DSM
    pipeline |= pdal.Writer.gdal(
        filename=dsm_tif, gdalopts="tiled=yes, compress=deflate", nodata=-9999,
        output_type=outfn, resolution=res
    )
    
    pipeline.execute()
    print(f"DSM saved to {dsm_tif}")
    tf = time.perf_counter() - ti 
    print(f"DSM saved to {tif}")
    print(f"Run time = {tf / 60:.2f} minute(s)")


def generate_dsm_by_return(laz, tif, res=10, outfn="mean"):
    ti = time.perf_counter()

    # Define the output file name
    tif = tif.replace('.tif', f"_returnDSM_{str(res)}_{outfn}.tif")

    # Check if the file already exists
    if os.path.isfile(tif):
        print(f"File already exists...\n{tif}")
        return tif 

    print("Extracting first return points and generating a DSM.")
    print(f"Generating DSM by return: {laz} \n-> {tif}")

    # Define the PDAL pipeline
    pipeline = pdal.Reader(laz)
    
    # Use the return number filter to only include the first returns
    pipeline |= pdal.Filter.range(limits="returnnumber[1:1]")
    
    # Write to GeoTIFF with specified resolution and compression
    pipeline |= pdal.Writer.gdal(
        filename=tif,
        gdalopts="tiled=yes,compress=deflate",
        nodata=-9999,
        output_type=outfn,
        resolution=res
    )

    # Execute the pipeline
    pipeline.execute()
    tf = time.perf_counter() - ti 
    print(f"DSM saved to {tif}")
    print(f"Run time = {tf / 60:.2f} minute(s)")
    return tif


def pointcloud_to_dsmtiff(laz,tif,lazo,res=10,outfn="mean",method="smrf",lc=0,hc=4,mode="range"):
    qi,pipeline = pdal_quick_info(laz)
    unique_classes = pdal_pc_classes(pipeline)
    # Check if 2 is in unique_classes
    if 2 in unique_classes:
        print('YES:::PointCloud ALREADY classified...')
        #generate_dtm(laz, tif, res=res,outfn=outfn)
        if mode == "range":
            generate_dsm_by_range(laz, tif, res=res, outfn=outfn,lc=lc,hc=hc)
        elif mode == "hag":
            generate_dsm_by_hag(lazo, tif, res, outfn)
        elif mode == "return":
            generate_dsm_by_return(laz, tif, res=res, outfn=outfn)
        else:
            print("mode not available...")
    else:
        print('NO:::PointCloud NOT YET classified...')
        lazo = clf_pointcloud(laz, lazo, method=method)
        #generate_dtm(lazo, tif, res=res,outfn=outfn)
        if mode == "range":
            generate_dsm_by_range(laz, tif, res=res, outfn=outfn,lc=lc,hc=hc)
        elif mode == "hag":
            generate_dsm_by_hag(lazo, tif, res, outfn)
        elif mode == "return":
            generate_dsm_by_return(laz, tif, res=res, outfn=outfn)
        else:
            print("mode not available...")

# already classified

In [12]:
res, outfn, method,lc,hc = 10, "max", "csf",0,4 # max
mode1, mode2, mode3 = "range","return","hag"

In [13]:
for i, location in enumerate(transect_names):
    if i > 0: break
    
    loc_laz_files = glob(f"{eba_laz_dir}/{location}/*.laz")
    loc_tif_dir = f"{eba_tif_dir}/{location}"
    loc_lazo_dir = f"{eba_lazclf_dir}/{location}"
    os.makedirs(loc_tif_dir, exist_ok=True)
    os.makedirs(loc_lazo_dir, exist_ok=True)
    print(f"{i}:{location} :: {len(loc_laz_files)} laz")
    for laz in loc_laz_files:
        btif = os.path.basename(laz).replace('.laz', '.tif')
        tif = f"{loc_tif_dir}/{btif}"
        lazo = f"{loc_lazo_dir}/{os.path.basename(laz)}"
        #pointcloud_to_dsmtiff(laz,tif,lazo,res,outfn,method,lc,hc,mode1) # very expensive @thrsh30min
        pointcloud_to_dsmtiff(laz,tif,lazo,res,outfn,method,lc,hc,mode2)
        #pointcloud_to_dsmtiff(laz,tif,lazo,res,outfn,method,lc,hc,mode3)


#pointcloud_to_dsmtiff(laz,tif,lazo,res=10,outfn="mean",method="smrf",lc=0,hc=4,mode="range")  

0:transectos_tocantins :: 1 laz
Unique classifications: [0 2 4]
YES:::PointCloud ALREADY classified...
Extracting first return points and generating a DSM.
Generating DSM by return: /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD/comprexn/transectos_tocantins/NP_T-0171.laz 
-> /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_returnDSM_10_max.tif
DSM saved to /media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/GeoTIFF/transectos_tocantins/NP_T-0171_returnDSM_10_max.tif
Run time = 0.50 minute(s)


In [30]:
# it changes the logic above, lets try to understand the classes of the new laz
# use all the othe classes except 2 

In [20]:
clf_laz_dir = "/media/ljp238/12TBWolf/ARCHIEVE/BrazilEBA/POINTCLOUD_CLF"

In [22]:
clf_laz_files = glob(f"{clf_laz_dir}/*/*.laz")
len(clf_laz_files)

20

In [31]:
# for laz_path in clf_laz_files:
#     print(os.path.basename(laz_path))
#     qi,pipeline = pdal_quick_info(laz_path)
#     unique_classes = pdal_pc_classes(pipeline)

In [27]:
laz_path = clf_laz_files[-4]

Unique classifications: [0 1 2]
